# db

### interface sql directly

In [17]:
import psycopg2
from heagVehicleLivedataUtils_sar23b.apiUtils import get_current_vehicle_data_as_dataframe

# Database connection settings
DB_SETTINGS = {
    'dbname': 'mydatabase',
    'user': 'user',
    'password': 'password',
    'host': 'localhost',  
    'port': 5432
}

def get_db_connection():
    conn = psycopg2.connect(**DB_SETTINGS)
    return conn

# test db connection
with get_db_connection() as conn, conn.cursor() as cursor:
    print("all fine")

In [ ]:
with get_db_connection() as conn, conn.cursor() as cursor:
    vehicle_line_list = get_current_vehicle_data_as_dataframe().reindex(columns=['lineId']).reset_index("vehicleId").values.tolist()
    for [vehicle, line] in vehicle_line_list:
        cursor.execute(f"INSERT INTO vehicledata (vehicleId, lineId) VALUES ({vehicle}, {line})")
    conn.commit()

In [9]:
with get_db_connection() as conn, conn.cursor() as cursor:
    cursor.execute("SELECT * FROM vehicleservice ()")
    trams = cursor.fetchall()
    
trams

[]

In [18]:
# add test relation
with get_db_connection() as conn, conn.cursor() as cursor:
    cursor.execute("INSERT INTO vehicleservice (time, vehicleid, lineid, category, direction, status, latitude, longitude, bearing, type) VALUES ('2024-11-30 22:43:57+01:00', 10, 7, 1, 'test', 3, 12.3, 3.14, 50, 1)")

### work with sqlalchemy

In [1]:
from sqlalchemy import create_engine
from heagVehicleLivedataUtils.apiUtils import get_current_vehicle_data_as_dataframe
import pandas as pd

# Database connection settings
DB_SETTINGS = {
    'dbname': 'mydatabase',
    'user': 'user',
    'password': 'password',
    'host': 'localhost',  
    'port': 5432
}

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**DB_SETTINGS))

# test connection
with engine.connect() as conn:
    print(pd.read_sql("SELECT * FROM vehicleservice", conn))
    #print(pd.read_sql("SELECT * FROM users", conn))
    

        identifier                 timestamp  vehicleid   lineid  category  \
0                1 2025-01-14 17:16:43+00:00        622      693         5   
1                2 2025-01-14 17:16:43+00:00         58        6         1   
2                3 2025-01-14 17:16:43+00:00        658  5140328         5   
3                4 2025-01-14 17:16:43+00:00        916  5729348         5   
4                5 2025-01-14 17:16:43+00:00        909  5729346         5   
...            ...                       ...        ...      ...       ...   
714725      714726 2024-10-02 17:59:51+00:00        973  4684866         5   
714726      714727 2024-10-02 17:59:51+00:00        435  5729345         5   
714727      714728 2024-10-02 17:59:51+00:00        104        9         1   
714728      714729 2024-10-02 17:59:51+00:00         16        9         1   
714729      714730 2024-10-02 17:59:51+00:00        439    10075         5   

                           direction  status   latitude  longit

In [68]:
# read contents of database
with engine.connect() as conn:
    print(pd.read_sql("SELECT * FROM vehicleservice", conn))

Empty DataFrame
Columns: [identifier, time, vehicleid, lineid, category, direction, status, latitude, longitude, bearing, type]
Index: []


#### test dataframe .to_sql

In [14]:
from heagVehicleLivedataUtils.vehicleDataUtils.read import vehicledata_index_names

df = get_current_vehicle_data_as_dataframe()
df.to_sql("vehicleservice",con=engine, if_exists='append', index_label=vehicledata_index_names)

158

In [15]:
df

lineid  category  \
timestamp                 vehicleid                      
2025-01-14 18:16:43+01:00 622            693         5   
                          58               6         1   
                          658        5140328         5   
                          916        5729348         5   
                          909        5729346         5   
...                                      ...       ...   
                          85               5         1   
                          292        4288610         5   
                          380        5729345         5   
                          61               9         1   
                          81               9         1   

                                                        direction  status  \
timestamp                 vehicleid                                         
2025-01-14 18:16:43+01:00 622              Fürth (Odenw.) Bahnhof       1   
                          58               Eberstadt Frankenstein       1   
                          658            Darmstadt Mathildenplatz       1   
                          916              Groß-Gerau Kreisklinik       1   
                          909        Worfelden Siedlung Hesselrod       1   
...                                                           ...     ...   
                          85              Darmstadt Böllenfalltor       1   
                          292            Ffm Flughafen Terminal 1       1   
                          380                   Darmstadt Schloss       1   
                          61              Darmstadt Böllenfalltor       1   
                          81              Darmstadt Böllenfalltor       1   

                                      latitude  longitude  bearing  type  
timestamp                 vehicleid                                       
2025-01-14 18:16:43+01:00 622        49.826037   8.831593      158     1  
                          58         49.920649   8.654409      178     1  
                          658        49.831382   8.833794      358     1  
                          916        49.911943   8.495504      269     1  
                          909        49.925237   8.559000      304     1  
...                                        ...        ...      ...   ...  
                          85         49.852471   8.670286      229     1  
                          292        49.871559   8.632008      151     1  
                          380        49.872980   8.652272       78     1  
                          61         49.862152   8.573268       78     1  
                          81         49.868504   8.662940      147     1  

[158 rows x 8 columns]

das problem war die indxe als seperate spalte nochmal mitzugeben

In [13]:
with engine.connect() as conn:
    df = pd.read_sql("SELECT * FROM vehicleservice", conn)

In [20]:
df[-2:].to_json(orient='records')

'[{"identifier":714729,"timestamp":1727891991000,"vehicleid":16,"lineid":9,"category":1,"direction":"Darmstadt B\\u00f6llenfalltor","status":1,"latitude":49.8718180556,"longitude":8.64398,"bearing":78,"type":1,"plausible":null},{"identifier":714730,"timestamp":1727891991000,"vehicleid":439,"lineid":10075,"category":5,"direction":"Darmstadt Hauptbahnhof","status":4,"latitude":49.8786869444,"longitude":8.6666522222,"bearing":270,"type":1,"plausible":null}]'